In [1]:
import pandas as pd
import numpy as np
import json, os
# Test single 2d lifting with COMETH
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
gui = nimble.NimbleGUI()
gui.serve(8080)
dataset_path="/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/DDPM/gt_2D3D/"
subject=9
action="Walking"#"Directions"
cameras=[60457274,54138969] # 54138969
cameras=[60457274,54138969,55011271,58860488] # 54138969

sequence_names=["S"+str(subject)+"_"+action+"."+str(camera) for camera in cameras]
# Load sequence and camera parameters
data = [pd.read_csv(os.path.join(dataset_path,sequence_name+".csv")) for sequence_name in sequence_names]

# Load camera parameters
with open(os.path.join("tmp/h36m/camera-parameters.json"), "r") as f:
    camera_params = json.load(f)
    Ks = [np.array(camera_params['intrinsics'][str(camera)]['calibration_matrix']) for camera in cameras]
    Rs = [np.array(camera_params['extrinsics']['S'+str(subject)][str(camera)]['R']) for camera in cameras]
    ts = [np.array(camera_params['extrinsics']['S'+str(subject)][str(camera)]['t'])/1000 for camera in cameras]
    # print(ts)

Web GUI serving on http://localhost:8080
GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070


In [2]:
# Load the human model and scale to correct dimension
s12 = Skeleton('BODY12.xml')
# Rajagopal
# s = DynamicSkeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml')
# BSM
s = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s_gt = DynamicSkeleton(config='BODY15_constrained_3D.xml')
kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
# gui.nativeAPI().renderSkeleton(s._nimble)

Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0
Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0


/home/emartini/COMETH/COMETH/DynamicSkeleton.py:205: RuntimeWarning: invalid value encountered in add
  s_avg = (self.q_l + self.q_u) / 2


In [3]:
# # Single camera camera view (only K required)
# t = 0
# gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
# kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]
# s12.load_from_numpy(gt[0].reshape(-1,3),kps)
# s.load_from_BODY12(s12)
# s.exact_scale()

# gui.nativeAPI().renderSkeleton(s._nimble)

# for t in range(0,2000):
#     gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
#     kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]
#     s.qpIK2D(kps2d[0],Ks[0],50,dt=100)
#     # # print(np.mean(gt[0][:,2]-s.to_numpy().reshape(-1,3)[:,2]))
#     gui.nativeAPI().renderSkeleton(s._nimble)
    
#     for i in range(0,int(gt[0].shape[0])):
#         pos = s.to_numpy()
#         gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.02,0.02,0.02]),pos=gt[0][i,:].squeeze(),color=np.array([0,255,0,0.8]))
#         gui.nativeAPI().createSphere(key=kps[i]+"_IK", radii=np.array([0.02,0.02,0.02]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,0.8]))
        
#         vec = gt[0][i,:].squeeze()
#         vec = 10 * vec / np.linalg.norm(vec)
#         gui.nativeAPI().createLine(key=kps[i]+"_line", points=[np.array([0,0,0]), vec], color=[1.0, 0.0, 0.0, 0.2])
        
        

In [4]:
# # Single camera in world coordinates RT
# t = 0
# gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
# kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]

# RT = np.vstack([np.hstack([Rs[0],ts[0]]),[0,0,0,1]])
# RT_i = np.linalg.inv(RT)
# R = RT_i[0:3,0:3]
# T = RT_i[0:3,3].reshape(-1,1)
# gt_w = (R@gt[0].transpose()+T).transpose()
# cam = T

# s12.load_from_numpy(gt_w,kps)
# s.load_from_BODY12(s12)

# s.exact_scale()

# gui.nativeAPI().renderSkeleton(s._nimble)#,overrideColor=np.array([0,255,0,0.8]),prefix='gt_1'

# for t in range(0,1000):
#     gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
#     gt_w = (R@gt[0].transpose()+T).transpose()
#     kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]
    
#     print("GT")
#     print(gt_w)
    
#     s.qpIK2Dw(kps2d[0],Ks[0],R,T,50,dt=100)
    
    
#     # s12.load_from_numpy(gt_w,kps)
#     # s.load_from_BODY12(s12)
#     # print(np.mean(gt[0][:,2]-s.to_numpy().reshape(-1,3)[:,2]))
#     gui.nativeAPI().renderSkeleton(s._nimble)
    
#     for i in range(0,int(gt[0].shape[0])):
#         pos = s.to_numpy()
#         # pos = (Rx@pos.transpose()).transpose().reshape(-1,1)
#         gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.02,0.02,0.02]),pos=gt_w[i,:].squeeze(),color=np.array([0,255,0,0.8]))
#         gui.nativeAPI().createSphere(key=kps[i]+"_IK", radii=np.array([0.02,0.02,0.02]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,0.8]))
        
#         vec = (cam.transpose()-gt_w[i,:]).squeeze()
        
#         # print("hey:",gt_w[i,:],cam.transpose(),vec)
        
#         # vec = -10 * vec / np.linalg.norm(vec)
#         gui.nativeAPI().createLine(key=kps[i]+"_line", points=[cam, gt_w[i,:]], color=[1.0, 0.0, 0.0, 0.5])

In [5]:
# Multi camera in world coordinates RT
t = 0
gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]

RT = [np.vstack([np.hstack([R,t]),[0,0,0,1]]) for R,t in zip(Rs,ts) ]
RT_i = [np.linalg.inv(R) for R in RT]
R = [r[0:3,0:3] for r in RT_i]
T = [r[0:3,3].reshape(-1,1) for r in RT_i]
gt_w = [(r@gt[i].transpose()+T[i]).transpose() for i,r in enumerate(R)]
cam = [t for t in T]

s12.load_from_numpy(gt_w[0],kps)

s.load_from_BODY12(s12)

s.exact_scale()

gui.nativeAPI().renderSkeleton(s._nimble)#,overrideColor=np.array([0,255,0,0.8]),prefix='gt_1'

for t in range(0,1000):
    gt = [np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000 for df in data]
    kps2d = [np.array([np.array(df.loc[t,[l+":U",l+":V"]].to_numpy()) for l in kps]).squeeze() for df in data]
    
    gt_w = [(r@gt[i].transpose()+T[i]).transpose() for i,r in enumerate(R)]
    
    s12.load_from_numpy(gt_w[0],kps)
    s_gt.load_from_BODY12(s12)
    s_gt.exact_scale(5)
    gui.nativeAPI().renderSkeleton(s_gt._nimble,overrideColor=np.array([0,255,0,0.15]),prefix='gt_')#,overrideColor=np.array([0,255,0,0.8]),prefix='gt_1' 
    
    # s12.load_from_numpy(gt_w,kps)
    # s.load_from_BODY12(s12)
    # print(np.mean(gt[0][:,2]-s.to_numpy().reshape(-1,3)[:,2]))
    
    s.multisource_qpIK2D(kps2d,Ks,R,T,cameras,100,dt=100)
    
    
    gui.nativeAPI().renderSkeleton(s._nimble)
    
    
    for j,gt_wcam in enumerate(gt_w):
        for i in range(0,int(gt[0].shape[0])):
            pos = s.to_numpy()
            # pos = (Rx@pos.transpose()).transpose().reshape(-1,1)
            gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.02,0.02,0.02]),pos=gt_wcam[i,:].squeeze(),color=np.array([0,255,0,0.8]))
            gui.nativeAPI().createSphere(key=kps[i]+"_IK", radii=np.array([0.02,0.02,0.02]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,0.8]))
            
            vec = (cam[j].transpose()-gt_wcam[i,:]).squeeze()
            
            # print("hey:",gt_wcam[i,:],cam.transpose(),vec)
            
            vec = (-8 * vec / np.linalg.norm(vec))+cam[j].transpose().squeeze()
            print(vec)
            gui.nativeAPI().createLine(key=kps[i]+"_line"+str(j), points=[cam[j], vec], color=[1.0, 0.0, 0.0, 0.15])
            # gui.nativeAPI().createLine(key=kps[i]+"_line"+str(j), points=[cam[j], gt_wcam[i,:]], color=[1.0, 0.0, 0.0, 0.15])

[ 1.14316059  3.39425826 -0.35561241]
[2.52784212 2.95898237 1.26296457]
[1.26389835 3.52224563 0.48422492]
[1.1998187  3.6273934  1.38043378]
[2.16503515 3.17295604 1.28927211]
[1.65578013 3.34643229 0.448734  ]
[0.6897588  3.80097683 1.24443565]
[0.26346462 3.91836385 1.18138885]
[ 1.61687749  3.18598663 -0.36847812]
[1.76265505 3.38379945 1.39057605]
[ 0.97264907  3.22551906 -1.08041177]
[ 1.50800816  3.00747331 -1.05861791]
[-1.21270608 -2.23555002  0.07680268]
[-0.05302579 -2.78405271  1.35885814]
[-1.20255782 -2.34140886  0.76774861]
[-1.28628168 -2.33854541  1.48030334]
[-0.42861177 -2.67244635  1.38689345]
[-0.87911    -2.4713449   0.70852767]
[-1.64837294 -2.16083654  1.37907495]
[-1.90122354 -2.02277753  1.34232893]
[-0.81479204 -2.38915799  0.00485429]
[-0.82387412 -2.53299266  1.48255888]
[-1.22940617 -2.06390326 -0.59133644]
[-0.7651766  -2.23616334 -0.68179922]
[-0.9989258   2.15472209  0.12535494]
[0.07893567 2.64039092 1.32113618]
[-1.01354449  2.24715374  0.76765548]
[

In [ ]:
K = Ks[0]
R = Rs[0]
T = ts[0]

# print(np.linalg.inv(K)@np.transpose(np.hstack([kps2d[0][0,:],[1]])))

# gt[0][0,2]
u = kps2d[0][0,0]
v = kps2d[0][0,1]
X = ((u-K[0,2])/K[0,0])#*gt[0][0,2]
Y = ((v-K[1,2])/K[1,1])#*gt[0][0,2]
# print(X)
# print(Y)
# print(np.array([X,Y,gt[0][0,2]]))
# print(gt[0][0,:])
# print(gt[0][0,0:3])

U = np.hstack([kps2d[0],np.ones((kps2d[0].shape[0],1))]).reshape(-1,1)
C = np.tile(np.array([K[0,2],K[1,2],0]),kps2d[0].shape[0]).reshape(-1,1)
F = np.tile(np.array([K[0,0],K[1,1],1]),kps2d[0].shape[0]).reshape(-1,1)

A = (U-C)/F

Z = np.repeat(gt[0][:,2],3).reshape(-1,1)

X = A*Z

# print(np.dot(R,gt[0][0,:]).reshape(-1,1)+T)

RT = np.vstack([np.hstack([R,T]),[0,0,0,1]])
RT_i = np.linalg.inv(RT)


R = RT_i[0:3,0:3]
T = RT_i[0:3,3].reshape(-1,1)


print(R@gt[0][0,:].reshape(-1,1)+T)
# print(R@gt[0][0,:].reshape(-1,1)+T)

# print(gt[0][0,:])

# From camera view to world view
np.zeros((3*kps2d[0].shape[0], 3*kps2d[0].shape[0]))

n = kps2d[0].shape[0]  # Change this to any positive integer

# Create the final (3n, 3n) matrix
R_e = np.zeros((3*n, 3*n))

# Place the 3x3 matrix on the diagonal
for i in range(n):
    R_e[i*3:(i+1)*3, i*3:(i+1)*3] = R

Te =  np.tile(T.transpose(),kps2d[0].shape[0]).reshape(-1,1)

# print(np.dot(R_e,X)+Te)

W = R_e@X+Te

# W.reshape(-1,3)
